In [1]:
aulas_desejadas = {"calculo1": 4, "fisica2": 6, "python": 8, "IA avançada": 10, "Sistemas lineares": 7}
lista_desejos = [4, 6, 8, 10, 7]


In [2]:
valor_creditos = {"calculo1": 9, "fisica2": 6, "python": 5, "IA avançada": 8, "Sistemas lineares": 7}
lista_creditos = [9, 6, 5, 8, 7]

In [3]:
limite_creditos = 16

In [4]:
import numpy as np

def sum_binary(bitstring, itens):
    peso = 0
    for n, i in enumerate(itens):
        if bitstring[n] == "1":
            peso += i
    return peso

In [5]:
itens = list(aulas_desejadas.keys())
n_itens = len(itens)
combinacoes = {}
valor_maximo = 0
solucao_otima = {}

In [6]:
for case_i in range(2**n_itens):  # Força bruta pelas 32 combinações
    combinacoes[case_i] = {}
    bitstring = np.binary_repr(case_i, n_itens)  # transforma um número em uma string binária
    combinacoes[case_i]["itens"] = [itens[n] for n, i in enumerate(bitstring) if i == "1"]
    combinacoes[case_i]["desejo"] = sum_binary(bitstring, lista_desejos)
    combinacoes[case_i]["creditos"] = sum_binary(bitstring, lista_creditos)
    if ( # teste pelo limite de créditos e as disciplinas mais desejadas
        combinacoes[case_i]["desejo"] >= valor_maximo  
        and combinacoes[case_i]["creditos"] <= limite_creditos 
    ):
        valor_maximo = combinacoes[case_i]["desejo"]
        solucao_otima = {
            "itens": combinacoes[case_i]["itens"],
            "desejo": combinacoes[case_i]["desejo"],
            "creditos": combinacoes[case_i]["creditos"],
        }

In [7]:
print(f"TA melhor combinação é {solucao_otima['itens']} ")
print(f"com os pesos das disciplinas: {solucao_otima['desejo']}")
print(f"concedendo os créditos {solucao_otima['creditos']} ")

TA melhor combinação é ['python', 'IA avançada'] 
com os pesos das disciplinas: 18
concedendo os créditos 13 


In [8]:
print(np.binary_repr(0, n_itens), combinacoes[0])
print(np.binary_repr(1, n_itens), combinacoes[1])
print(np.binary_repr(2, n_itens), combinacoes[2])
print()
print(solucao_otima)

00000 {'itens': [], 'desejo': 0, 'creditos': 0}
00001 {'itens': ['Sistemas lineares'], 'desejo': 7, 'creditos': 7}
00010 {'itens': ['IA avançada'], 'desejo': 10, 'creditos': 8}

{'itens': ['python', 'IA avançada'], 'desejo': 18, 'creditos': 13}


In [9]:
Q = -np.diag(list(aulas_desejadas.values()))  # matriz para os desejados.
x_opt = np.array([[1 if i in solucao_otima["itens"] else 0] for i in aulas_desejadas.keys()])  
print(x_opt.T)
opt_str = "".join(str(i[0]) for i in x_opt)
print(opt_str)
custo_minimo = (x_opt.T @ Q @ x_opt)[0, 0]
print(Q)
print(custo_minimo)

[[0 0 1 1 0]]
00110
[[ -4   0   0   0   0]
 [  0  -6   0   0   0]
 [  0   0  -8   0   0]
 [  0   0   0 -10   0]
 [  0   0   0   0  -7]]
-18


In [10]:
print(np.log2(limite_creditos))
N = round(np.ceil(np.log2(limite_creditos)))  
print(N)
pesos = list(valor_creditos.values()) + [2**k for k in range(N)]
print(pesos)
QT = np.pad(Q, ((0, N), (0, N))) 
print(QT)
n_qubits = len(QT)
print(valor_creditos.values())

4.0
4
[9, 6, 5, 8, 7, 1, 2, 4, 8]
[[ -4   0   0   0   0   0   0   0   0]
 [  0  -6   0   0   0   0   0   0   0]
 [  0   0  -8   0   0   0   0   0   0]
 [  0   0   0 -10   0   0   0   0   0]
 [  0   0   0   0  -7   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]]
dict_values([9, 6, 5, 8, 7])


In [11]:
lambd = 2  # Lambda suficiente para os elementos acima da diagonal sempre serem preenchidos
# Recalculando todos os valores da Matriz
print(QT)
for i in range(n_qubits):
    print(lambd * pesos[i])
    print((pesos[i] - 2 * limite_creditos))
    QT[i, i] += lambd * pesos[i] * (pesos[i] - 2 * limite_creditos)  
    for j in range(i + 1, n_qubits):
        QT[i, j] += 2 * lambd * pesos[i] * pesos[j]  
offset = lambd * limite_creditos**2
print(QT)

[[ -4   0   0   0   0   0   0   0   0]
 [  0  -6   0   0   0   0   0   0   0]
 [  0   0  -8   0   0   0   0   0   0]
 [  0   0   0 -10   0   0   0   0   0]
 [  0   0   0   0  -7   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]]
18
-23
12
-26
10
-27
16
-24
14
-25
2
-31
4
-30
8
-28
16
-24
[[-418  216  180  288  252   36   72  144  288]
 [   0 -318  120  192  168   24   48   96  192]
 [   0    0 -278  160  140   20   40   80  160]
 [   0    0    0 -394  224   32   64  128  256]
 [   0    0    0    0 -357   28   56  112  224]
 [   0    0    0    0    0  -62    8   16   32]
 [   0    0    0    0    0    0 -120   32   64]
 [   0    0    0    0    0    0    0 -224  128]
 [   0    0    0    0    0    0    0    0 -384]]


In [12]:
slack_string = np.binary_repr(limite_creditos - solucao_otima["creditos"], N)[::-1]
print(slack_string)
x_opt_slack = np.concatenate(
    (x_opt, np.array([[int(i)] for i in slack_string]))
)  # combinando a string de busca
print(x_opt_slack)
opt_str_slack = "".join(str(i[0]) for i in x_opt_slack)
print(opt_str_slack)
cost = (x_opt_slack.T @ QT @ x_opt_slack)[0, 0] + offset  # Calculo do custo total
print(f"Custo:{cost}")

1100
[[0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]]
001101100
Custo:-18
